# Querys 

_Este ejercicio está pensado para hacerse en Pandas y/o SQL._ 

Carga el archivo https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2020/2020-01-14/passwords.csv en un DataFrame __SIN DESCARGARTE EL ARCHIVO__

1. Crea una columna llamada “decrypt_time_seconds” y transforma los valores de “value” a segundos. Encontrarás el tipo en el que está en la columna “time_unit”
2. Responde a estás preguntas
    1. ¿Cuántas contraseñas empiezan por la secuencia 123?
    2. ¿Cuál es el tiempo medio en horas necesario para descifrar estas contraseñas que empiezan por 123? ¿Cómo se compara con la media de todas las contraseñas del conjunto de datos?
    3. ¿Cuántas contraseñas no contienen un número?
    4. ¿Cuántas contraseñas contienen al menos un número?
    5. ¿Existe una diferencia evidente en el tiempo de descifrado en línea entre las contraseñas que no contienen ningún número y las que contienen al menos un número?
    6. ¿Cuántas contraseñas contienen al menos una de las siguientes puntuaciones: [.!?\-] (pista: recuerde que este conjunto de datos contiene contraseñas débiles...)?
    7. ¿Qué contraseñas de los conjuntos de datos tardaron menos tiempo en descifrarse mediante decriptación online? ¿Cuál tardó más?
    

Para resolver este ejercicio puedes crear todas las columnas que quieras si eso te ayuda para realizar las querys de forma sencilla. 

En todas las preguntas adjunta la query que has utilizado (tanto en Pandas o SQL).  

### Puntos extra:

haz el ejercicio 2.a, 2.c,2.d,2.f con expresiones regulares.



In [21]:
import pandas as pd

In [22]:
# usamos la url para leer el csv
url = 'https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2020/2020-01-14/passwords.csv'

In [23]:
df = pd.read_csv(url)

In [24]:
df

,rank,password,category,value,time_unit,offline_crack_sec,rank_alt,strength,font_size
0,1.0,password,password-related,6.91,years,2.170000e+00,1.0,8.0,11.0
1,2.0,123456,simple-alphanumeric,18.52,minutes,1.110000e-05,2.0,4.0,8.0
2,3.0,12345678,simple-alphanumeric,1.29,days,1.110000e-03,3.0,4.0,8.0
3,4.0,1234,simple-alphanumeric,11.11,seconds,1.110000e-07,4.0,4.0,8.0
4,5.0,qwerty,simple-alphanumeric,3.72,days,3.210000e-03,5.0,8.0,11.0
...,...,...,...,...,...,...,...,...,...
502,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
503,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
504,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
505,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
# Para eliminar los nulos
df_sin_nan = df.dropna()

1. Crea una columna llamada “decrypt_time_seconds” y transforma los valores de “value” a segundos. Encontrarás el tipo en el que está en la columna “time_unit”

In [27]:
# Para ver los valores unicos en la columna time_unit, que queremos modificar a segundos en la nueva columna
df_sin_nan.time_unit.unique()

array(['years', 'minutes', 'days', 'seconds', 'months', 'weeks', 'hours'],
      dtype=object)

In [35]:
# Lo mas sencillo es crear una funcion que tenga todas las trasnformaciones a segundos para luego aplicarla
def conversion(fila):
    conversion_factors = {'years': 365 * 24 * 60 * 60,
        'months': 30 * 24 * 60 * 60,
        'weeks': 7 * 24 * 60 * 60,
        'days': 24 * 60 * 60,
        'hours': 60 * 60,
        'minutes': 60,
        'seconds': 1}

    unidad = fila['time_unit'] # para saber de que columna obtenemos la unidad
    factor = conversion_factors.get(unidad, 0) # para obtener el valor del diccionario. Si no está seria cero, aunque no hay ningun caso asi en este dataframe

    return fila['value'] * factor

# y ya podemos crear la nueva columna utilizando apply y una función lambda
df_sin_nan['decrypt_time_seconds'] = df.apply(lambda fila: conversion(fila), axis=1) # a cada fila le aplicamos el factor correspondiente

df_sin_nan['decrypt_time_seconds']
 

C:\Users\Picar\AppData\Local\Temp\ipykernel_19528\3697146649.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sin_nan['decrypt_time_seconds'] = df.apply(lambda fila: conversion(fila), axis=1) # a cada fila le aplicamos el factor correspondiente


0      2.179138e+08
1      1.111200e+03
2      1.114560e+05
3      1.111000e+01
4      3.214080e+05
           ...     
495    3.214080e+05
496    2.179138e+08
497    8.268480e+06
498    3.214080e+05
499    2.909827e+09
Name: decrypt_time_seconds, Length: 500, dtype: float64

2. Responde a estás preguntas
    1. ¿Cuántas contraseñas empiezan por la secuencia 123?

In [37]:
# CON PANDAS
df_123 = df_sin_nan[df_sin_nan['password'].apply(lambda x: str(x).startswith('123'))] # PARA USAR STARTSWITH HAY QUE USAR UN APPLY, Y CON EL LAMBDA NOS ASEGURAMOS QUE SE REVISE TODA LA COLUMNA

In [38]:
df_123

,rank,password,category,value,time_unit,offline_crack_sec,rank_alt,strength,font_size,decrypt_time_seconds
1,2.0,123456,simple-alphanumeric,18.52,minutes,1.110000e-05,2.0,4.0,8.0,1111.20
2,3.0,12345678,simple-alphanumeric,1.29,days,1.110000e-03,3.0,4.0,8.0,111456.00
3,4.0,1234,simple-alphanumeric,11.11,seconds,1.110000e-07,4.0,4.0,8.0,11.11
5,6.0,12345,simple-alphanumeric,1.85,minutes,1.110000e-06,6.0,4.0,8.0,111.00
23,24.0,1234567,simple-alphanumeric,3.09,hours,1.110000e-04,24.0,4.0,8.0,11124.00
48,49.0,123456789,simple-alphanumeric,1.84,weeks,1.110000e-02,49.0,4.0,8.0,1112832.00
57,58.0,123123,simple-alphanumeric,18.52,minutes,1.110000e-05,58.0,7.0,11.0,1111.20
413,414.0,123321,simple-alphanumeric,18.52,minutes,1.110000e-05,415.0,4.0,8.0,1111.20
462,463.0,123abc,simple-alphanumeric,3.70,weeks,2.240000e-02,465.0,32.0,23.0,2237760.00


2. ¿Cuál es el tiempo medio en horas necesario para descifrar estas contraseñas que empiezan por 123? ¿Cómo se compara con la media de todas las contraseñas del conjunto de datos?

In [45]:
# ME LO PIDE EN HORAS Y LO TENEMOS EN SEGUNDOS, PODEMOS HACER UNA NUEVA TRANSFORMACION Y CREAR OTRA COLUMNA, ESTA VEZ MAS SENCILLA
df_123['tiempo_horas'] = df_123['decrypt_time_seconds'].apply(lambda x: x/60/60)

C:\Users\Picar\AppData\Local\Temp\ipykernel_19528\3712842408.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_123['tiempo_horas'] = df_123['decrypt_time_seconds'].apply(lambda x: x/60/60) # Dado que son segundos lo dividimos entre 60 (minutos) y entre 24(horas)


In [46]:
df_123['tiempo_horas']

1        0.308667
2       30.960000
3        0.003086
5        0.030833
23       3.090000
48     309.120000
57       0.308667
413      0.308667
462    621.600000
Name: tiempo_horas, dtype: float64

In [47]:
df_123['tiempo_horas'].mean() # 107,30 horas de media, hay tres contraseñas que cambian mucho la media

107.30332438271606

In [48]:
# Buscamos la media de todo el dataframe original
df_sin_nan['tiempo_horas'] = df_sin_nan['decrypt_time_seconds'].apply(lambda x: x/60/60)

C:\Users\Picar\AppData\Local\Temp\ipykernel_19528\485942351.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sin_nan['tiempo_horas'] = df_sin_nan['decrypt_time_seconds'].apply(lambda x: x/60/60)


In [49]:
df_sin_nan['tiempo_horas'].mean() # 13917.565247227776 es la media, MUCHISIMO MAS ALTA QUE LA DE LAS QUE EMPIEZAN POR 1,2,3. ES DECIR, ES MAS DIFICIL DESENCRIPTARLAS

13917.565247227776

3. ¿Cuántas contraseñas no contienen un número?

In [52]:
# Nos filtra el Dataframe con las contraseñas QUE NO TIENEN NINGUN NUMERO
df_SIN_numero = df_sin_nan[~df_sin_nan['password'].str.contains('\d')] 
# Estamos usando la virgulilla para que sea AL CONTRARIO (QUE NO TENGA NUMERO, QUE ES LO QUE SIGNIFICA ('\d'), CUALQUIER DIGITO ), al final ya estamos usando expresion regular

In [53]:
df_SIN_numero 

,rank,password,category,value,time_unit,offline_crack_sec,rank_alt,strength,font_size,decrypt_time_seconds,tiempo_horas
0,1.0,password,password-related,6.91,years,2.17000,1.0,8.0,11.0,217913760.0,60531.60
4,5.0,qwerty,simple-alphanumeric,3.72,days,0.00321,5.0,8.0,11.0,321408.0,89.28
6,7.0,dragon,animal,3.72,days,0.00321,7.0,8.0,11.0,321408.0,89.28
7,8.0,baseball,sport,6.91,years,2.17000,8.0,4.0,8.0,217913760.0,60531.60
8,9.0,football,sport,6.91,years,2.17000,9.0,7.0,11.0,217913760.0,60531.60
...,...,...,...,...,...,...,...,...,...,...,...
494,495.0,hotrod,cool-macho,3.72,days,0.00321,497.0,7.0,11.0,321408.0,89.28
495,496.0,reddog,cool-macho,3.72,days,0.00321,498.0,6.0,10.0,321408.0,89.28
496,497.0,alexande,name,6.91,years,2.17000,499.0,9.0,12.0,217913760.0,60531.60
497,498.0,college,nerdy-pop,3.19,months,0.08350,500.0,7.0,11.0,8268480.0,2296.80


In [58]:
df_SIN_numero.shape # 446 contraseñas no contienen ningun numero

(446, 11)

4. ¿Cuántas contraseñas contienen al menos un número?

In [50]:
# Filtramos el Dataframe para que nos muestre SOLO LOS ELEMENTOS DE LA COLUMNA PASSWORD QUE TENGAN AL MENOS UN NUMERO
df_con_numero = df_sin_nan[df_sin_nan['password'].str.contains('\d')]

In [51]:
df_con_numero

,rank,password,category,value,time_unit,offline_crack_sec,rank_alt,strength,font_size,decrypt_time_seconds,tiempo_horas
1,2.0,123456,simple-alphanumeric,18.52,minutes,1.110000e-05,2.0,4.0,8.0,1.111200e+03,0.308667
2,3.0,12345678,simple-alphanumeric,1.29,days,1.110000e-03,3.0,4.0,8.0,1.114560e+05,30.960000
3,4.0,1234,simple-alphanumeric,11.11,seconds,1.110000e-07,4.0,4.0,8.0,1.111000e+01,0.003086
5,6.0,12345,simple-alphanumeric,1.85,minutes,1.110000e-06,6.0,4.0,8.0,1.110000e+02,0.030833
11,12.0,696969,simple-alphanumeric,18.52,minutes,1.110000e-05,12.0,1.0,4.0,1.111200e+03,0.308667
12,13.0,abc123,simple-alphanumeric,3.70,weeks,2.240000e-02,13.0,32.0,23.0,2.237760e+06,621.600000
18,19.0,111111,simple-alphanumeric,18.52,minutes,1.110000e-05,19.0,0.0,0.0,1.111200e+03,0.308667
19,20.0,2000,simple-alphanumeric,11.11,seconds,1.110000e-07,20.0,0.0,0.0,1.111000e+01,0.003086
23,24.0,1234567,simple-alphanumeric,3.09,hours,1.110000e-04,24.0,4.0,8.0,1.112400e+04,3.090000
25,26.0,trustno1,simple-alphanumeric,92.27,years,2.902000e+01,26.0,25.0,20.0,2.909827e+09,808285.200000


In [57]:
df_con_numero.shape # 54 contraseñas

(54, 11)

5. ¿Existe una diferencia evidente en el tiempo de descifrado en línea entre las contraseñas que no contienen ningún número y las que contienen al menos un número?

In [55]:
# PODEMOS HACER LA MEDIA DE DESCIFRADO DEL DATAFRAME FILTRADO CON NUMERO Y EL SIN NUMERO, usamos la columna tiempo en horas que para mi es mas claro
df_SIN_numero['tiempo_horas'].mean()


8095.36166367713

In [56]:
df_con_numero['tiempo_horas'].mean() # las que tienen al menos un numero tienen un tiempo de descifrado mayor

62004.65410396089

 6. ¿Cuántas contraseñas contienen al menos una de las siguientes puntuaciones: [.!?\-] (pista: recuerde que este conjunto de datos contiene contraseñas débiles...)?

In [64]:
# creamos una variable con los distintos signos
signos = '[.!?\-]'
df_raras = df_sin_nan[df_sin_nan['password'].str.contains(signos)]

In [65]:
df_raras  # no hay ninguna

,rank,password,category,value,time_unit,offline_crack_sec,rank_alt,strength,font_size,decrypt_time_seconds,tiempo_horas,signos


   7. ¿Qué contraseñas de los conjuntos de datos tardaron menos tiempo en descifrarse mediante decriptación online? ¿Cuál tardó más?

In [66]:
print(df_123['tiempo_horas'].mean())
print(df_con_numero['tiempo_horas'].mean())
print(df_SIN_numero['tiempo_horas'].mean())
# Las contraseñas que mas cuesta desencriptar de manera online son las que contienen al menos un numero y las que menos las que empiezan por 123

107.30332438271606
62004.65410396089
8095.36166367713
